In [1]:
## Effiecient use of Autoencoder for novelty detection
# Stacked Autoencoder so far gained a huge popularity in dimension reduction and novelty(outlier/unusual pattern) mapping.
# dataset I will use is a glimpse of real world data that was live almost five years ago. 
# Main objective is to obtain the reconstruction error using stacked auto encoder.

In [2]:
import numpy as np
import pandas as pd
from _ExpectationMaximization_ import SingleseriesGaussianMixture
from keras.models import Sequential,Model
from keras.layers import Dense
import data_processing

Using TensorFlow backend.


In [3]:
# Here I am using two custom library ( _ExpectationMAximization_ and data_processing). Entire code of  _ExpectationMAximization_ is available on https://github.com/jm2909/Expectation_maximization 

In [4]:
dataread = data_processing.Datareading(dataframe = 'dataset.csv')
Independent_data,Response = dataread.dataprocessed()

In [5]:
Independent_data.head(5)

,Unnamed: 0,Shareholders_Funds,Secured_Loans,Debt___Loan_Funds,Liabilities,Sundry_Debtors,Inventories,Cash_and_Bank_Balance,Current_Assets,Current_Liabilities,Sales_Turnover___Operating_Incom,Net_Sales,Income,Operating_Profit
0,0,8443.04,0.0,35.50,8598.40,484.35,1188.60,91.60,2169.50,3773.26,13240.71,11796.83,11916.13,1503.35
1,1,8235.61,0.0,0.00,8351.55,410.71,1255.59,304.30,2369.06,3784.21,13108.18,11738.21,12017.77,1775.54
2,2,7824.84,32.0,35.03,7966.01,397.22,1121.47,503.38,2400.93,3620.31,12471.74,11149.61,11428.75,1914.46
3,3,7382.80,157.0,163.06,8019.31,303.45,1133.55,678.38,2467.47,3390.00,12639.44,11357.96,11602.76,2125.02
4,4,7192.27,500.0,510.73,8198.32,187.74,1099.54,1652.56,3287.16,3272.61,10700.02,9660.29,9946.59,2112.63


In [6]:
Response.head(5)

0    0
1    0
2    0
3    0
4    0
Name: Rating, dtype: int64

In [8]:
Response.astype(object).value_counts()

0    5073
1     605
Name: Rating, dtype: int64

In [9]:
scaler,Xind  = data_processing.__Processing__(df = Independent_data,process='min-max')


In [10]:
Xind[:5]

array([[  0.00000000e+00,   8.44035463e-02,   0.00000000e+00,
          7.76475734e-05,   2.35309258e-02,   1.65677595e-02,
          1.83312544e-02,   1.84915258e-03,   1.71414480e-02,
          3.94041229e-02,   2.63142874e-02,   2.46140780e-02,
          2.49297844e-02,   8.97226191e-02],
       [  1.76149375e-04,   8.35840012e-02,   0.00000000e+00,
          0.00000000e+00,   2.30505344e-02,   1.40488170e-02,
          1.93644116e-02,   6.14204447e-03,   1.87181926e-02,
          3.95184737e-02,   2.60509003e-02,   2.44917674e-02,
          2.51390123e-02,   9.50835235e-02],
       [  3.52298749e-04,   8.19610704e-02,   3.09318131e-04,
          7.66195632e-05,   2.23002404e-02,   1.35873757e-02,
          1.72959379e-02,   1.01600460e-02,   1.89700008e-02,
          3.78068673e-02,   2.47860539e-02,   2.32636539e-02,
          2.39265031e-02,   9.78196153e-02],
       [  5.28448124e-04,   8.02145934e-02,   1.51759208e-03,
          3.56653896e-04,   2.24039668e-02,   1.03798630e-0

In [11]:
# simple min-max preprocessing I have done above.

In [12]:
# I have made it a stacked encoder, comprised of two relu layers of 10 and 5 respectively.
# Defining parameters
batch_size = 17
nb_epochs  =10
hidden_dim1 = 10
hidden_dim2 = 5

In [13]:
# Creating Model
model = Sequential()
model.add(Dense(hidden_dim1,input_dim = Xind.shape[1],activation = 'relu',name= 'Encoder1'))
model.add(Dense(hidden_dim2,activation = 'relu',name= 'Encoder2'))
model.add(Dense(Xind.shape[1], activation= 'sigmoid',name = 'Decoder'))
model.compile(loss='mean_squared_error', optimizer='adadelta')


In [14]:
# fitting model
model.fit(Xind,Xind,epochs=nb_epochs,batch_size=batch_size,verbose = 2)

Epoch 1/10
0s - loss: 0.1133
Epoch 2/10
0s - loss: 0.0081
Epoch 3/10
0s - loss: 0.0058
Epoch 4/10
0s - loss: 0.0043
Epoch 5/10
0s - loss: 0.0031
Epoch 6/10
0s - loss: 0.0024
Epoch 7/10
0s - loss: 0.0021
Epoch 8/10
0s - loss: 0.0020
Epoch 9/10
0s - loss: 0.0020
Epoch 10/10
0s - loss: 0.0020


In [15]:
# slight overfitting thoough after epoch 8 but for experiment we can ignore it for now.
# i have used relu just because of kind of normalizer I have used and of course fast learning.

In [16]:
#Let's have prediction and reconstruction error
predict = model.predict(Xind, batch_size=17)
Anomaly = predict - Xind

In [17]:
Anomaly[:5]

array([[ 0.0799773 ,  0.00015037,  0.00774089,  0.00954829, -0.0118004 ,
         0.00016575, -0.00740168,  0.01532422, -0.00369491, -0.03452536,
        -0.01774482, -0.0150985 , -0.00204436, -0.02740159],
       [ 0.08028117,  0.00065877,  0.00766733,  0.00954379, -0.01140103,
         0.00253712, -0.00851564,  0.01084851, -0.00536764, -0.03467328,
        -0.01753812, -0.01504242, -0.00249716, -0.03288969],
       [ 0.08055019,  0.00205136,  0.00730601,  0.00940962, -0.01070995,
         0.00288927, -0.00650446,  0.00668821, -0.00569388, -0.03298363,
        -0.01630913, -0.01386653, -0.00146656, -0.03571717],
       [ 0.08104959,  0.0036576 ,  0.00606149,  0.00909128, -0.01084558,
         0.00601335, -0.00672454,  0.00303613, -0.00626426, -0.0305861 ,
        -0.01665807, -0.01432835, -0.00198833, -0.0398465 ],
       [ 0.0807895 ,  0.00371376,  0.00260752,  0.0081709 , -0.01138633,
         0.0097055 , -0.00637542, -0.01692952, -0.01296663, -0.02944651,
        -0.01293292, -0.01

In [18]:
# My next objective was to study the distribution of mean(recontruction) of each row.
# If any particular attribute is far towards natural then mean value must deviate from normal.
# although it is a perfect scenario to apply Multivariate Gaussian Cluster, but for simplicity I have used univariate here.

In [19]:
# suppose I want to have 8 clusters
# i.e. making assumption that series is mixed up of 8 distinc distribution (assuming normal for our case)
n_cluster = 8
em = SingleseriesGaussianMixture(np.mean(Anomaly,1),n_cluster)
# mu_array and sigma_array will be mean and sigma of respective univariate normal distribution
# Sum_array is the sum  explained in each cluster
# Count is the count of observation in each cluster
#sumsq is the sum_of_square explained in each cluster.
# lables is the fitted cluster labels.
mu_array, sigma_array, sum_array, count, sumsq,labels = em.__EM__()


In [20]:
mu_array

array([-0.00078031,  0.00252016,  0.01048909, -0.1189585 , -0.00802826,
       -0.08387662, -0.07372571, -0.0924562 ])

In [21]:
sigma_array

array([ 0.00110098,  0.00206457,  0.00331361,  0.11616115,  0.00603722,
        0.0004205 ,  0.00488365,  0.00485236])

In [22]:
# Next objective was to check how many Default and Non-Default are covered by each cluster.

In [23]:
rate= []
c= []
for  i in range(0,n_cluster):
    if len(Response.loc[labels.icol(0) == i].astype(object).value_counts().index) == 1 and (Response.loc[labels.icol(0) == i].astype(object).value_counts().index[0] == 0):
        zero = Response.loc[labels.icol(0) == i].astype(object).value_counts()[0]
        one =1
    elif len(Response.loc[labels.icol(0) == i].astype(object).value_counts().index) == 1 and (Response.loc[labels.icol(0) == i].astype(object).value_counts().index[0] == 1):
        one = Response.loc[labels.icol(0) == i].astype(object).value_counts()[0]
        zero =1
    else:
        zero = Response.loc[labels.icol(0) == i].astype(object).value_counts()[0]
        one = Response.loc[labels.icol(0) == i].astype(object).value_counts()[1]
    ratio = zero/one
    rate.append(ratio)
    c.append(zero+one)



Report_Reconstruction_Error=pd.DataFrame(np.transpose(np.vstack((mu_array,sigma_array,np.array(rate),np.array(c)))),columns  = ['Mean','Sigma','Odds','Count'])
Report_Reconstruction_Error =Report_Reconstruction_Error.sort(['Mean'])

print(Report_Reconstruction_Error)

       Mean     Sigma  Odds   Count
3 -0.118958  0.116161  41.0   212.0
7 -0.092456  0.004852  19.0    20.0
5 -0.083877  0.000420   4.0     5.0
6 -0.073726  0.004884  29.0    30.0
4 -0.008028  0.006037   7.0   825.0
0 -0.000780  0.001101   9.0   717.0
1  0.002520  0.002065   7.0  2995.0
2  0.010489  0.003314  10.0   877.0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [24]:
## that's interesting. before explaining let me define Odds. Odds is nothing but  count of zero / count of one
## I have sorted it, so rows are not arranged. 
## In group 7, ratio of Non-default: default is 19 against 1, same for group 5 and 6 are 4 against 1 and 29 against 1 approxminately.

In [ ]:
# And for group 3 we have approximately 5 defaults and 205 non-defaults. 
# If we apply logistic regression then these observation during training will affect the analysis
# beacuse it seems like they are not actually 1, may be thought as an outlier of zero or rare pattern.
# Thus helps to detect important observations.